In [1]:
import trimesh
import numpy as np
import open3d as o3d
import copy
import torch
import os
import glob
import ot
import random

In [2]:
seed = 0

# Python
random.seed(seed)

# NumPy
np.random.seed(seed)

# PyTorch
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
kitti_root = "KITTI-Sequence"

folders = sorted(os.listdir(kitti_root))

point_clouds = []

for folder in folders:
    folder_path = os.path.join(kitti_root, folder)
    
    if os.path.isdir(folder_path):    
        points_file = os.path.join(folder_path, f"{folder}_points.obj")
        
        if os.path.exists(points_file):
            points = np.array(trimesh.load(points_file).vertices)
            point_clouds.append(points)
        

print(f"Total de nuvens de pontos carregadas: {len(point_clouds)}")

ground_truth = np.load('ground_truth.npy')

Total de nuvens de pontos carregadas: 30


In [4]:
for cloud in point_clouds[:3]:
    print(f"Forma da nuvem de pontos: {cloud.shape}")
    print(f"Primeiros 5 pontos:\n{cloud[:5]}")
    print(f"Ultimos 5 pontos:\n{cloud[-5:]}")

Forma da nuvem de pontos: (62553, 3)
Primeiros 5 pontos:
[[-9.110238 18.638599  0.909355]
 [-9.151207 18.5732    0.908344]
 [-9.206173 18.536804  0.908331]
 [-9.278134 18.536415  0.909315]
 [-9.080147 17.786371  0.883326]]
Ultimos 5 pontos:
[[ 1.455129  3.812988 -1.764532]
 [ 1.445153  3.822563 -1.767544]
 [ 1.438176  3.840138 -1.773556]
 [ 1.419202  3.825716 -1.76457 ]
 [ 1.507196  4.092375 -1.895561]]
Forma da nuvem de pontos: (62340, 3)
Primeiros 5 pontos:
[[-9.056182 17.949978  0.888338]
 [-9.106149 17.908581  0.887326]
 [-9.184107 17.92219   0.889309]
 [-9.03208  17.089552  0.862305]
 [-9.042068 17.043352  0.8603  ]]
Ultimos 5 pontos:
[[ 1.454139  3.826792 -1.770537]
 [ 1.448161  3.846375 -1.778548]
 [ 1.426188  3.824943 -1.765563]
 [ 1.42221   3.849532 -1.775574]
 [ 1.414232  3.863128 -1.780585]]
Forma da nuvem de pontos: (62138, 3)
Primeiros 5 pontos:
[[-8.969279 17.366747  0.869398]
 [-8.996253 17.284346  0.867389]
 [-9.042223 17.239948  0.866377]
 [-9.102187 17.221554  0.86736

In [5]:
print(ground_truth[:3])

[[[ 1.000000e+00  9.043680e-12  2.326809e-11  5.551115e-17]
  [ 9.043683e-12  1.000000e+00  2.392370e-10  3.330669e-16]
  [ 2.326810e-11  2.392370e-10  9.999999e-01 -4.440892e-16]
  [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]

 [[ 9.999978e-01  5.272628e-04 -2.066935e-03 -4.690294e-02]
  [-5.296506e-04  9.999992e-01 -1.154865e-03 -2.839928e-02]
  [ 2.066324e-03  1.155958e-03  9.999971e-01  8.586941e-01]
  [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]

 [[ 9.999910e-01  1.048972e-03 -4.131348e-03 -9.374345e-02]
  [-1.058514e-03  9.999968e-01 -2.308104e-03 -5.676064e-02]
  [ 4.128913e-03  2.312456e-03  9.999887e-01  1.716275e+00]
  [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]]


In [6]:
def find_nearest_neighbors_knn(source, target_points):
    point_cloud_tree = o3d.geometry.KDTreeFlann(source)
    source_points = np.asarray(source.points)
    
    source_corr = np.zeros_like(target_points)
    distances = np.zeros(len(target_points))
    
    for i, point in enumerate(target_points):
        _, idx, dist = point_cloud_tree.search_knn_vector_3d(point, 1)
        source_corr[i] = source_points[idx[0]]
        distances[i] = dist[0]
        
    return source_corr, distances

In [7]:
# Carregar e preparar nuvens de pontos
source = o3d.geometry.PointCloud()
target = o3d.geometry.PointCloud()
source.points = o3d.utility.Vector3dVector(point_clouds[0])
target.points = o3d.utility.Vector3dVector(point_clouds[1])

# Colorir para visualização
source.paint_uniform_color([0.5, 0.5, 0.5])  # Cinza
target.paint_uniform_color([0, 0, 1])        # Azul

# Alinhamento inicial baseado nos centróides
source_points = np.asarray(source.points)
target_points = np.asarray(target.points)
source_centroid = np.mean(source_points, axis=0)
target_centroid = np.mean(target_points, axis=0)
initial_transform = np.eye(4)
initial_transform[:3, 3] = target_centroid - source_centroid
source = source.transform(initial_transform)

# Parâmetros do ICP
max_iterations = 50
threshold = 1e-6
prev_error = float('inf')
curr_error = 1000

In [8]:
patience = 0
consecutive_small_improvements = 0

# Loop principal do ICP
for iteration in range(max_iterations):
    # Verificar convergência com critério mais inteligente
    improvement = prev_error - curr_error
    
    # Se a melhoria for pequena
    if improvement < threshold:
        consecutive_small_improvements += 1
        patience += 1
        
        # Punir mais se o erro estiver aumentando
        if curr_error > prev_error:
            patience += 1
            print(f"Aviso: Erro aumentou na iteração {iteration + 1}")
        
        # Verificar convergência com critérios dinâmicos
        if patience >= 5 or consecutive_small_improvements >= 7:
            print(f"Convergência atingida após {iteration + 1} iterações")
            break
    else:
        # Reduzir paciência gradualmente em vez de zerar completamente
        patience = max(0, patience - 1)
        consecutive_small_improvements = 0
    
    prev_error = curr_error
    
    # 1. Encontrar correspondências
    source_corr, distances = find_nearest_neighbors_knn(source, target_points)
    
    # 2. Calcular transformação rígida ótima
    source_centroid = np.mean(source_corr, axis=0)
    target_centroid = np.mean(target_points, axis=0)
    
    source_centered = source_corr - source_centroid
    target_centered = target_points - target_centroid
    
    # 3. Calcular matriz de covariância e SVD
    cov = target_centered.T @ source_centered
    U, _, Vt = np.linalg.svd(cov)
    
    # Garantir rotação própria (det = 1)
    S = np.eye(3)
    if np.linalg.det(U @ Vt) < 0:
        S[2, 2] = -1
    
    # 4. Calcular rotação e translação
    R = U @ S @ Vt
    t = target_centroid - R @ source_centroid
    
    # 5. Aplicar transformação
    transform = np.eye(4)
    transform[:3, :3] = R
    transform[:3, 3] = t
    source = source.transform(transform)
    
    # 6. Atualizar erro
    curr_error = np.mean(distances)
    print(f"Iteração {iteration + 1}, Erro: {curr_error:.6f}")

# print("Transformação final:")
# print(transform)


Iteração 1, Erro: 0.065812
Iteração 2, Erro: 0.063219
Iteração 3, Erro: 0.061734
Iteração 4, Erro: 0.060764
Iteração 5, Erro: 0.060133
Iteração 6, Erro: 0.059594
Iteração 7, Erro: 0.058912
Iteração 8, Erro: 0.057905
Iteração 9, Erro: 0.056340
Iteração 10, Erro: 0.054273
Iteração 11, Erro: 0.051626
Iteração 12, Erro: 0.048323
Iteração 13, Erro: 0.044664
Iteração 14, Erro: 0.041145
Iteração 15, Erro: 0.038054
Iteração 16, Erro: 0.035288
Iteração 17, Erro: 0.033093
Iteração 18, Erro: 0.031455
Iteração 19, Erro: 0.030219
Iteração 20, Erro: 0.029204
Iteração 21, Erro: 0.028418
Iteração 22, Erro: 0.027763
Iteração 23, Erro: 0.027178
Iteração 24, Erro: 0.026672
Iteração 25, Erro: 0.026268
Iteração 26, Erro: 0.025952
Iteração 27, Erro: 0.025702
Iteração 28, Erro: 0.025522
Iteração 29, Erro: 0.025392
Iteração 30, Erro: 0.025297
Iteração 31, Erro: 0.025231
Iteração 32, Erro: 0.025185
Iteração 33, Erro: 0.025153
Iteração 34, Erro: 0.025129
Iteração 35, Erro: 0.025110
Iteração 36, Erro: 0.025096
I

In [ ]:
def inverse_transform(T):
    """Inverte uma transformação homogênea 4x4"""
    R = T[:3, :3]
    t = T[:3, 3]
    
    # Calcular inversa
    R_inv = R.T  # Inversa da rotação é a transposta
    t_inv = -R_inv @ t  # Translação inversa
    
    # Construir matriz inversa
    T_inv = np.eye(4)
    T_inv[:3, :3] = R_inv
    T_inv[:3, 3] = t_inv
    
    return T_inv

# Após calcular sua transformação final no ICP:
transform_invertido = inverse_transform(transform)
print("Transformação invertida (comparável ao ground truth):")
print(transform_invertido)

Transformação invertida (comparável ao ground truth):
[[ 1.00000000e+00  1.90334688e-06  2.51593207e-06  2.59647326e-05]
 [-1.90333583e-06  1.00000000e+00 -4.39224058e-06  3.72013557e-04]
 [-2.51594043e-06  4.39223579e-06  1.00000000e+00 -3.29806316e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [10]:
erro = np.linalg.norm(transform_invertido - ground_truth[0])
print(f"Erro em relação ao ground truth: {erro}")

Erro em relação ao ground truth: 0.0003744522329037734


In [11]:
# Visualização da nuvem de pontos alinhada
source_temp = copy.deepcopy(source)
target_temp = copy.deepcopy(target)
source_temp.paint_uniform_color([1, 0.706, 0])
target_temp.paint_uniform_color([0, 0.651, 0.929])
source_temp.transform(transform)
o3d.visualization.draw_geometries([source_temp, target_temp], zoom=0.4459, front=[0.9288, -0.2951, -0.2242], lookat=[1.6784, 2.0612, 1.4451], up=[-0.3402, -0.9189, -0.1996])

In [12]:
ground_truth[0]

array([[ 1.000000e+00,  9.043680e-12,  2.326809e-11,  5.551115e-17],
       [ 9.043683e-12,  1.000000e+00,  2.392370e-10,  3.330669e-16],
       [ 2.326810e-11,  2.392370e-10,  9.999999e-01, -4.440892e-16],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])